In [ ]:
import numpy as np
import sys
# sys.path.insert(0, '/Users/pouyabashivan/Dropbox (MIT)/Codes/dldata')
import dldata.metrics.utils as utils
import dldata.stimulus_sets.hvm as hvm
import cPickle
import h5py
import sys
import os
from sklearn.cross_validation import KFold
from sklearn import cross_decomposition
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
np.random.seed([0])
from dldata.physiology.hongmajaj.mappings import CHANNEL_INFO
from sklearn.preprocessing import scale
%matplotlib inline

import seaborn as sns
from scipy.misc import imread

np.random.seed(123)
import tensorflow.contrib.slim as slim

%matplotlib inline
from tensorflow.python.framework import ops
from tensorflow.python.ops import gen_nn_ops       
import scipy.stats as stats

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

sns.set_style("white")
sns.set_style("ticks", {"xtick.major.size": 14, "ytick.major.size": 14})
sns.set_context("poster")


dataset_hvm = hvm.HvMWithDiscfade()
meta_hvm = dataset_hvm.meta
neural_fea = np.array(dataset_hvm.neuronal_features)


# Utility Functions

In [ ]:
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr
from scipy.interpolate import griddata, interp2d


def reps_to_array(reps):
    """
    reps: dictionary containing the reps for each variation level  
    """
    max_reps = np.max([reps[i].shape[0] for i in reps.keys()], axis=0)
    hvm_neural = np.zeros((max_reps, 5760, reps['V0'].shape[2]))
    hvm_neural[...] = np.NaN

    c = 0
    for key in reps:
        shape = reps[key].shape
        hvm_neural[:shape[0], c:c+shape[1], :] = reps[key]
        c += shape[1]
    return hvm_neural


def concat_reps(rep_list):
    """
    """
    max_reps = np.max([r.shape[0] for r in rep_list])
    resized_reps = []
    for r in rep_list:
        tmp = np.zeros((max_reps, r.shape[1], r.shape[2]))
        tmp[...] = np.NaN
        tmp[:r.shape[0], :, :] = r
        resized_reps.append(tmp)
    return np.concatenate(resized_reps, axis=-1)

def fix_nan_reps(reps):
    """Some of the entries in neural reps might be nan. 
    Substitute those values by the average response of 
    corresponding neurons to all images over all valid reps.
    reps = [n_reps, n_samples, n_neurons]
    """
    if np.any(np.isnan(reps)):
        # find the indices of nan neurons
        nan_ind = np.isnan(reps)
        _, _, nan_neu_ind = np.nonzero(nan_ind)

        corrected_reps = reps
        for n in np.unique(nan_neu_ind):
            # create a mask of all nan values for a neuron
            mask = np.zeros(shape=nan_ind.shape, dtype=bool)
            mask[:, :, n] = True
            masked_nan_ind = nan_ind & mask

            # substitue all nan values of neuron by average neuron response
            av_neuron_act = np.nanmean(reps[:, :, n])
            corrected_reps[masked_nan_ind] = av_neuron_act
        return corrected_reps
    else:
        return reps

def project_reps(input_reps, W_mat):
    """Project each rep of neural data using the projection matrix
    input_reps = [n_reps, n_samples, n_neurons]"""
    input_reps = fix_nan_reps(input_reps)
    reps = []
    for rep in input_reps:
        reps.append(scale(rep))
    comp_reps = np.tensordot(reps, W_mat, axes=1)
    return comp_reps

def fit_reg(X, Y):
    """
    Fits a linear regression model to the data and returns regression model with score and predictions"""
    reg = LinearRegression()
    reg.fit(X, Y)
    preds = reg.predict(X)
    score = pearsonr(Y, preds)
    return reg, score, preds

def predict_outputs(features, weights):
    """
    Predict outputs given input features and weights."""
    model_pcs = np.matmul(features - weights['pca_b'], weights['pca_w'])
    preds = np.matmul(model_pcs, weights['pls_w']) + weights['pls_b'] 
    return preds


def resize_mat(mat, new_size):
    """
    Resize a matrix to the desired size. Input size is [num_channels, num_pixels, num_pixels]"""
    if mat.ndim==2:
        mat = np.expand_dims(mat, axis=0)
    num_ch, _, num_pix = np.array(mat).shape

    x = np.arange(0, num_pix)
    y = np.arange(0, num_pix)
    ratio = (new_size - 1.) / (num_pix - 1)

    x_new = np.arange(0, new_size)
    y_new = np.arange(0, new_size)

    output = []
    for i in range(num_ch):
        resized_rf_func = interp2d(x * ratio, y * ratio, mat[i], kind='cubic')
        tmp_out = resized_rf_func(x_new, y_new)
        output.append(tmp_out)

    return np.squeeze(output)    


In [ ]:
# Load Left Magneto data and ID lookup dictionary (corresponding ids from hvm 5760 images)
from scipy.io import loadmat

data = loadmat('/braintree/home/bashivan/dropbox/Data/Ko_data/V4_data/mag_left_v4.mat')
images_lookup = loadmat('/braintree/home/bashivan/dropbox/Data/Ko_data/id_object_lookup.mat')['id_object_lookup']
# site_locations = loadmat('/braintree/home/bashivan/dropbox/Data/Ko_data/V4_data/kk_2_pb_sites.mat')

# Find matching image ids on HVM
ids = []
for r in images_lookup:
    image_num = int(np.nonzero(meta_hvm['id'] == r[1])[0])
    ids.append(image_num)


# Separable (Mask-Mix) Mapping

In [ ]:
from sklearn.decomposition import PCA
import h5py
from scipy.stats import pearsonr
from cubemap.sep_map import SeparableMap

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

map_type = 'separable'   # 'linreg' or 'separable'

features_h5_hvm = h5py.File('/braintree/data2/active/users/bashivan/model_features/alexnet_retina299_input299_out299.h5')
# features_h5_coco = h5py.File('/braintree/data2/active/users/bashivan/model_features/alexnet_coco_retina299_input299_out299.h5')

features_hvm = np.array(features_h5_hvm['mdl_conv3'])[ids, :]
# features_coco = np.array(features_h5_hvm['mdl_conv3'])[:640, :]

# feats = np.concatenate((features_hvm, features_coco), axis=0)

feats = features_hvm
neurons = np.nanmean(v4_data_reps, axis=0)

if map_type == 'linreg':
    X = pca_feats
else:
    X = feats
Y = scale(neurons)
    
num_images = X.shape[0]
np.random.seed(0)
train_ind = np.random.choice(range(num_images), num_images * 9 // 10, replace=False)
test_ind = np.nonzero(~ np.in1d(range(num_images), train_ind))[0]

all_scores = np.zeros((3, 3))
# consistency = np.array(analyst.compute_consistency(v4_data_reps, 20, population=False)).mean(1)

for ls_ind, ls in enumerate([0.4]): 
    for ld_ind, ld in enumerate([0.3]):
        print('Mapping with ls={}, ld={}'.format(ls, ld))
        mapper = SeparableMap(graph=None, max_epochs=700, tol=0.1, 
                              init_lr=0.01, batch_size=50, ls=ls, ld=ld, num_neurons=num_select_cells,
                              map_type=map_type, inits=None, decay_rate=200, log_rate=100
                             )
        mapper.fit(X[train_ind], Y[train_ind])
        preds = mapper.predict(X[test_ind])
        scores = np.array([pearsonr(preds[:, i], Y[test_ind, i])[0] for i in range(preds.shape[-1])])
        all_scores[ls_ind, ld_ind] = np.median(scores)
        
        print('Scores: ', np.median(scores))
#         print('Consistency: ', np.median(consistency)) 
        print('Norm. EV: ', np.median(scores ** 2 / consistency ** 2))

print(all_scores)